In [1]:
import xgboost as xgb
import shap
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from tqdm import tqdm
import os
from itertools import product
from sklearn.cluster import KMeans

In [2]:
from M_GWXGB_1 import GeoWeightedXGBoostTrainer, GeoWeightedXGBoostInterpreter, GeoWeightedXGBoostPredictor, M_GWXGB_Initial, OptimizedBandwidth, M_GXGB


In [81]:
from M_GWXGB_cluster import GeoWeightedXGBoostTrainer_1, GeoWeightedXGBoostInterpreter, GeoWeightedXGBoostPredictor

In [3]:
import math
import time

In [4]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [5]:
from sklearn.neighbors import NearestNeighbors

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 25
mpl.rcParams['axes.unicode_minus'] = 'False'

# Function defination

## Adj_R2

In [7]:
def adjusted_r2_score(y_true, y_pred, n_features):
    n_samples = len(y_true)
    r2 = r2_score(y_true, y_pred)
    
    if n_samples <= n_features + 1:
        raise ValueError(" ")
    
    adjusted_r2 = 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)
    return adjusted_r2

## Plotting

In [8]:
size = 50
def moving_wind(f1_est,X1,w=2):
    mat = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            if i == 0 and j == 0:
                top = f1_est.reshape(size,size)[0:w,0:w]
                bot = X1.reshape(size,size)[0:w,0:w]
                
            if i == 0 and j > 0:
                top = f1_est.reshape(size,size)[0:i+w,j-1:j+w]
                bot = X1.reshape(size,size)[0:i+w,j-1:j+w]
                
            if j == 0 and i > 0:
                top = f1_est.reshape(size,size)[i-1:i+w,0:j+w]
                bot = X1.reshape(size,size)[i-1:i+w,0:j+w]
                
            if i > 0 and j > 0 : 
                top = f1_est.reshape(size,size)[i-1:i+w,j-1:j+w]
                bot = X1.reshape(size,size)[i-1:i+w,j-1:j+w]
            
            m = np.linalg.lstsq(bot.reshape(-1,1), top.reshape(-1,1), rcond=None)[0][0,0]
            if m > size:
                print(i,j)
            mat[i,j] = m
            
    return mat

In [9]:
from palettable.matplotlib import Inferno_3

def plot_1(b,title,vmin=None,vmax=None):
    size = 50
    plt.figure(figsize=(6, 4), dpi=300)
    plt.tight_layout()
    plt.imshow(b.reshape(size,size),vmin=vmin, vmax=vmax, cmap=Inferno_3.mpl_colormap)
    plt.title(title,fontsize=25)
    plt.colorbar()
    plt.xticks([])
    plt.yticks([])

# Data generation

In [8]:
size = 50

X1 = np.random.uniform(-1.5, 1.5, size*size)
X2 = np.random.uniform(0, 3, size*size)
X3 = np.random.uniform(-1.5, 1.5, size*size)
X4 = np.random.uniform(-1.5, 1.5, size*size)
X = np.vstack([X1, X2, X3, X4]).T

u = np.array([np.linspace(0, size-1, num=size)]*size).reshape(-1)
v = np.array([np.linspace(0, size-1, num=size)]*size).T.reshape(-1)
coords = list(zip(u, v))

f0 = 2
f1 = 2 * X1
f2 = np.log(X2 / 2) * X2
f3 = X3**3

f4 = np.zeros_like(X4)
mask_left = u < (size / 2)
mask_right = ~mask_left

f4[mask_left] = X4[mask_left] ** 3
f4[mask_right] = -X4[mask_right] ** 3

err = np.random.uniform(-1.5,1.5,size*size)
fs = np.vstack([f1, f2, f3, f4]).T
y = (f0 + f1 + f2 + f3 + f4 + err).reshape(-1, 1)

In [46]:
data = pd.read_csv(r'E:\PaperCode\S-MGXGB\SimuResults_S2_1\data_1.csv')
y = np.array(data['y']).reshape(-1, 1)
X = np.array(data[['x1', 'x2', 'x3', 'x4']]).reshape(2500, 4)
coords = np.array(data[['lng', 'lat']]).reshape(2500, 2)

In [ ]:
## input the figure saving path

save_path= r'...\\Simu_path'

In [ ]:
## actual parameter surfaces

for i in range(4):

    fig = plt.figure(figsize = (8, 8))

    plot_1(fs[:, i] / X[:, i], fr'actual: $f_{i+1} $')
    

    plt.savefig(save_path + '\\' + fr'actual_X{i+1}_s.jpg', 
                dpi = 300)

In [ ]:
## actual non-linear relationships

for i in range(3):
    plt.figure(figsize=(9, 7))
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    plt.ylabel(rf'$f_{i+1} X_{i+1} \sim X_{i+1}$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')
    
    plt.savefig(save_path + '\\' + rf'true_f{i+1}_nl.jpg',
                dpi = 300)

In [ ]:
## actual non-linear relationships of f4

plt.figure(figsize=(9, 7))

plt.scatter(X[:, 3], 
            fs[:, 3], 
            c = fs[:, 3],
            cmap = Inferno_3.mpl_colormap, 
            s = 3, 
            alpha = 0.7)

plt.ylabel(rf'$f_4 X_4 \sim X_4$')
plt.xlabel(fr'$\mathrm{{X}}_4$')

plt.savefig(save_path + '\\' + rf'true_f4_nl.jpg',
            dpi = 300)

In [ ]:
plt.figure(figsize=(7, 7))

plt.scatter(X4[mask_right], 
            f4[mask_right], 
            c = f4[mask_right],
            cmap = Inferno_3.mpl_colormap, 
            s = 3, 
            alpha = 0.7)

plt.ylabel(rf'$f_4 X_4 \sim X_4$')
plt.xlabel(fr'$\mathrm{{X}}_4$')

plt.savefig(save_path + '\\' + rf'true_f4_nl_right.jpg',
            dpi = 300)

# XGBoost model
5 fold cross-validation is used to form a complete prediction

In [9]:
class XGBoostTrainer_1:
    def __init__(self, n_splits=5, random_state=42, n_jobs=-1):
        self.best_model = None
        self.best_params = None
        self.oof_predictions = None
        self.cv_scores = []
        self.n_splits = n_splits
        self.random_state = random_state
        self.best_iterations_per_fold = []
        self.oof_shap_values = None
        self.n_jobs = n_jobs

    def _train_fold(self, params, X, y, train_idx, val_idx, fold_num):

        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        dtrain_fold = xgb.DMatrix(X_train_fold, label=y_train_fold)
        dvalid_fold = xgb.DMatrix(X_val_fold, label=y_val_fold)

        watchlist = [(dvalid_fold, 'eval')]

        model = xgb.train(params,
                         dtrain_fold,
                         num_boost_round=1000,
                         evals=watchlist,
                         early_stopping_rounds=30,
                         verbose_eval=False)

        return {
            'fold_num': fold_num,
            'score': model.best_score,
            'model': model,
            'val_idx': val_idx
        }

    def objective(self, params, X, y):
        params['max_depth'] = int(params['max_depth'])
        params['min_child_weight'] = int(params['min_child_weight'])

        param_use = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            **params
        }

        kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        folds = list(kf.split(X, y))

        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self._train_fold)(
                param_use, X, y, train_idx, val_idx, fold_num
            )
            for fold_num, (train_idx, val_idx) in enumerate(folds)
        )

        cv_scores = [res['score'] for res in results]
        avg_cv_score = np.mean(cv_scores)
        
        return {'loss': avg_cv_score, 'status': STATUS_OK}

    def _cv_fold(self, X, y, train_idx, val_idx, fold_num):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        dtrain_fold = xgb.DMatrix(X_train_fold, label=y_train_fold)
        dvalid_fold = xgb.DMatrix(X_val_fold, label=y_val_fold)

        watchlist = [(dtrain_fold, 'train'), (dvalid_fold, 'eval')]

        fold_model = xgb.train(self.best_params,
                             dtrain_fold,
                             num_boost_round=2000,
                             evals=watchlist,
                             early_stopping_rounds=50,
                             verbose_eval=100 if fold_num == 0 else False)

        best_iteration = fold_model.best_iteration
        fold_score = fold_model.best_score

        oof_preds_fold = fold_model.predict(dvalid_fold, iteration_range=(0, best_iteration))
        
        fold_explainer = shap.TreeExplainer(fold_model)
        fold_shap_value = fold_explainer.shap_values(dvalid_fold)

        return {
            'fold_num': fold_num,
            'best_iteration': best_iteration,
            'score': fold_score,
            'val_idx': val_idx,
            'oof_pred': oof_preds_fold,
            'shap_values': fold_shap_value,
            'model': fold_model
        }

    def calcu_oof_and_shap(self, X, y, tune=True, max_evals=50):
        if y.ndim > 1 and y.shape[1] == 1:
            y = y.ravel()
            
        if X.ndim == 1:
            X = X.reshape(-1, 1)

        if tune or self.best_params is None:
            self.tune_params(X, y, max_evals=max_evals)
        elif not tune and self.best_params is None:
            raise ValueError("Cannot train without tuning if best_params are not already set.")

        print(f"\nStarting {self.n_splits}-Fold CV for OOF predictions (parallel)...")
        kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        folds = list(kf.split(X, y))

        self.oof_predictions = np.zeros(X.shape[0])
        self.oof_shap_values = np.zeros(X.shape)
        self.cv_scores = []
        self.best_iterations_per_fold = []

        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self._cv_fold)(X, y, train_idx, val_idx, fold_num)
            for fold_num, (train_idx, val_idx) in enumerate(folds)
        )

        for res in results:
            self.best_iterations_per_fold.append(res['best_iteration'])
            self.cv_scores.append(res['score'])
            self.oof_predictions[res['val_idx']] = res['oof_pred']
            self.oof_shap_values[res['val_idx']] = res['shap_values']

        oof_rmse = np.sqrt(mean_squared_error(y, self.oof_predictions))
        oof_r2 = adjusted_r2_score(y, self.oof_predictions, X.shape[1])
        
        print("\n--- Training Final Model on Full Data ---")
        final_num_boost_round = int(np.median(self.best_iterations_per_fold))
        dfull = xgb.DMatrix(X, label=y)

        self.best_model = xgb.train(self.best_params,
                                  dfull,
                                  num_boost_round=final_num_boost_round,
                                  verbose_eval=100)

        print("Final model training complete.")
        return self.best_model, self.oof_predictions, oof_rmse, oof_r2, self.oof_shap_values

    def tune_params(self, X, y, max_evals=50):
        search_space = {
            "max_depth": hp.quniform('max_depth', 3, 8, 1),
            "learning_rate": hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
            "subsample": hp.uniform('subsample', 0.6, 1.0),
            "colsample_bytree": hp.uniform('colsample_bytree', 0.6, 1.0),
            "min_child_weight": hp.quniform('min_child_weight', 1, 6, 1),
            "gamma": hp.uniform('gamma', 0, 0.5),
            "reg_alpha": hp.loguniform('reg_alpha', np.log(0.001), np.log(1.0)),
            "reg_lambda": hp.loguniform('reg_lambda', np.log(0.1), np.log(10.0)),
        }

        trials = Trials()
        best = fmin(
            fn=lambda params: self.objective(params, X, y),
            space=search_space,
            algo=tpe.suggest,
            max_evals=max_evals,
            trials=trials,
            rstate=np.random.default_rng(self.random_state)
        )

        self.best_params = {
            'max_depth': int(best['max_depth']),
            'learning_rate': best['learning_rate'],
            'subsample': best['subsample'],
            'colsample_bytree': best['colsample_bytree'],
            'min_child_weight': int(best['min_child_weight']),
            'gamma': best['gamma'],
            'reg_alpha': best['reg_alpha'],
            'reg_lambda': best['reg_lambda'],
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'random_state': self.random_state
        }
        print("\nBest parameters found:")
        print(self.best_params)
        return self.best_params

    def predict(self, X):
        dmatrix = xgb.DMatrix(X)
        return self.best_model.predict(dmatrix)

    def compute_shap_values(self, X):
        explainer = shap.TreeExplainer(self.best_model)
        dmatrix = xgb.DMatrix(X)
        return explainer.shap_values(dmatrix)

## Train model

In [ ]:
start_time = time.time()

X_1 = np.concatenate((X, coords), axis=1)
xgb_trainer = XGBoostTrainer_1(n_splits=5, random_state=42, n_jobs=4)
final_model, oof_preds, oof_rmse, oof_r2, oof_shap = xgb_trainer.calcu_oof_and_shap(X_1, y, tune=True, max_evals=50)

print(f'rmse: {oof_rmse}')
print(f'r2: {oof_r2}')

end_time = time.time()
elapsed_time = end_time - start_time

print(elapsed_time)

## Model evaluation

In [ ]:
print("RMSE: ", oof_rmse)
print("adj_R2: ", adjusted_r2_score(oof_predictions, y, 4))

## Plotting

### Non-linear

In [ ]:
## global relationships
locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-3.2, -0.9, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                oof_shap[:, i],
                c = 'none',
                edgecolors = 'darkred',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(oof_shap[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['XGB curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'xgb_X{i+1}_nl.jpg', 
                dpi = 300)

In [ ]:
## local

x_test = np.random.uniform(-1.5, 1.5, 2500)
f_test = -x_test ** 3

point_id = 'your point id'

for i in range(4):
    plt.figure(figsize=(9, 7))
    b = oof_shap[:, i][point_id] / X[:, i][point_id]
    plt.scatter(x_test, 
                b*x_test,
                c = 'none',
                edgecolors = 'green',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(x_test, 
                f_test, 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(b*x_test, f_test)
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['XGB curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'xgb_X{i+1}_nl_local_2448.jpg', 
                dpi = 300)
    
    plt.close()

### Spatial

In [ ]:
vmins = [1.8, 0.5, -3, -2.25]
vmaxs = [2.2, 3, 3, 2.25]

for i in range(4):

    fig = plt.figure(figsize = (8, 8))

    plot_1(moving_wind(oof_shap[:, i], X[:, i]), fr'XGB: $f_{i+1} $', vmin = vmins[i], vmax = vmaxs[i])
    
    expected = fs[:, i] / X[:, i].reshape(-1)
    predicted = moving_wind(oof_shap[:, i], X[:, i]).reshape(-1)
    cosine_sim = np.dot(predicted, expected)/(np.linalg.norm(predicted) * np.linalg.norm(expected))
    
    print(f'the cosine similarity index for x_{i+1} is {cosine_sim}')

    plt.savefig(save_path + '\\' + fr'xgb_X{i+1}_s.jpg', 
                dpi = 300)

plt.close()

# GWXGB model

## train model

obtain the best model from 19 bandwidth (using the geographic clustering approach for searching the best bandwidth)

In [34]:
def evaluate_test_set(target, gwxgb_trainer):
    mse_scores = []
    r2_scores = []
    test_indices_list, test_predictions_list = gwxgb_trainer.test_indices_list, gwxgb_trainer.test_predictions_list
    for i, (test_indices, y_pred_test) in enumerate(zip(test_indices_list, test_predictions_list)):
            
        y_test = target[test_indices]
        mse_score = mean_squared_error(y_test, y_pred_test)
        mse_scores.append(mse_score)
        r2 = r2_score(y_test, y_pred_test)
        r2_scores.append(r2)
    return np.mean(mse_scores), np.mean(r2_scores) if mse_scores else None

In [ ]:
## 寻找最优带宽

k_range = (125, 275, 125)
save_dir = r'your output path'

gwxgb_trainer = GeoWeightedXGBoostTrainer(data=X,
                                          target=y,
                                          locations=coords,
                                          n_jobs=8, 
                                          use_full_sample = True,
                                          n_clusters=100)                          
optimal_k = gwxgb_trainer.search_optimal_k_nearest(k_range)
print(optimal_k)
gwxgb_trainer.fit()

local_y_hat, y_hat = gwxgb_trainer.predict()


In [47]:
def spatial_interpolator_fill(y_hat, coords, method='weighted', k=10, power=2):
    """
    使用空间邻居的预测值填补缺失值
    
    参数：
    - y_hat_df: pd.DataFrame，包含 'index' 和 'predy' 的模型预测结果（只有部分样本有）
    - coords: ndarray[n_samples, 2]，所有样本点的坐标
    - method: str，可选 'mean' 或 'weighted'
    - k: int，使用多少个邻居来进行插值
    - power: int，weighted 模式下的距离幂次，默认为2
    
    返回：
    - y_pred_full: ndarray[n_samples]，完整预测向量
    """
	
    y_hat_df = pd.DataFrame(y_hat).reset_index()
    y_hat_df.columns = ['ori_idx', 'predy']
    
    n_samples = coords.shape[0]
    y_pred_full = np.full(n_samples, np.nan)

    known_indices = y_hat_df['ori_idx'].values
    y_pred_full[known_indices] = y_hat_df['predy'].values

    known_coords = coords[known_indices]
    known_preds = y_hat_df['predy'].values

    # 构建 KNN 模型用于寻找最近邻（只考虑有预测值的点）
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(known_coords)

    # 找到所有缺失点
    all_indices = np.arange(n_samples)
    missing_indices = np.setdiff1d(all_indices, known_indices)
    missing_coords = coords[missing_indices]

    # 找出每个缺失点的 k 个邻居
    distances, indices = nbrs.kneighbors(missing_coords)

    for i, idx in enumerate(missing_indices):
        neighbor_preds = known_preds[indices[i]]
        if method == 'mean':
            y_pred_full[idx] = np.mean(neighbor_preds)
        elif method == 'weighted':
            d = distances[i]
            # 防止距离为0
            d[d == 0] = 1e-6
            weights = 1 / np.power(d, power)
            y_pred_full[idx] = np.sum(weights * neighbor_preds) / np.sum(weights)
        else:
            raise ValueError("method must be 'mean' or 'weighted'")

    return y_pred_full

In [78]:
feature_name = ['X1', 'X2', 'X3', 'X4']

start_time = time.time()

gxgb_trainer_known_k = GeoWeightedXGBoostTrainer_1(
    data=X,
    target=y,
    locations=coords,
    k_nearest=250,           # 已知的最佳带宽
    n_clusters=100,          # 使用100个聚类中心
    n_splits=5,              # 5折交叉验证
    use_full_sample=False,   # 关键：启用基于聚类的OOF模式
    n_jobs=8,
    tune_evals=30            # 每个局部模型的调参次数
)

# 2. 调用 .fit() 进行训练
# lightweight_mode=False (默认) 确保执行的是完整的OOF训练，而不是轻量级搜索
gxgb_trainer_known_k.fit()

end_time_1 = time.time()

print(f'fitting time {end_time_1 - start_time}')
# 3. 评估模型性能
# evaluate() 会自动使用OOF预测结果进行评估，因为 use_full_sample=False
print("\n--- Evaluating OOF Performance ---")
mse, r2 = gxgb_trainer_known_k.evaluate()
if mse is not None:
    print(f"Final OOF Mean Squared Error: {mse:.4f}")
    print(f"Final OOF R-squared: {r2:.4f}")

# 4. 获取OOF预测结果
# 预测结果是一个DataFrame，索引是样本编号，值为OOF预测值
oof_predictions_df = gxgb_trainer_known_k.prediction()
print("\nOOF Predictions Head:")
print(oof_predictions_df.head())

end_time_2 = time.time()
print(f'fitting time {end_time_2 - start_time}')


--- Starting Final Model Fitting with k_nearest = 250 ---
Mode: OOF. Using 100 cluster centers.


D:\Anaconda\Path\envs\py390\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Training strategy: K-Fold CV (OOF)
fitting time 653.9310035705566

--- Evaluating OOF Performance ---
Evaluating using OOF predictions...
Evaluation - MSE: 1.1493, R²: 0.8484
Final OOF Mean Squared Error: 1.1493
Final OOF R-squared: 0.8484


NameError: name 'gwr_trainer_known_k' is not defined

In [82]:
# 初始的 k_nearest 值不重要，因为它会在搜索过程中被覆盖
gwr_trainer_search = GeoWeightedXGBoostTrainer_1(
    data=X,
    target=y,
    locations=coords, 
    n_clusters=100,
    n_splits=5,
    use_full_sample=False,   # 模式仍然是基于聚类
    n_jobs=8,
    tune_evals=15            # 在搜索阶段可以减少调参次数以加快速度
)

# --- 步骤 2.1: 搜索最佳 k_nearest ---
# 定义一个搜索范围，例如从100到500，步长为50
k_range_to_search = (100, 500, 50)
print(f"\n--- Step 2.1: Searching for optimal k_nearest in range {k_range_to_search} ---")
optimal_k = gwr_trainer_search.search_optimal_k_nearest(k_range=k_range_to_search)

# search_optimal_k_nearest 方法会自动将找到的最佳k值设置回实例中
# gwr_trainer_search.k_nearest 现在已经是 optimal_k
print(f"\nOptimal k_nearest found and set to: {gwr_trainer_search.k_nearest}")

# --- 步骤 2.2: 使用找到的最佳k进行完整的OOF训练 ---
print("\n--- Step 2.2: Performing full OOF training with optimal k ---")
# 如果需要，可以增加调参次数以获得更精细的模型
gwr_trainer_search.tune_evals = 30 

# 直接在同一个实例上调用 .fit() 即可
# 此时，它将使用 optimal_k 和完整的OOF模式进行训练
gwr_trainer_search.fit() 

# 3. 评估和获取结果 (同场景1)
print("\n--- Evaluating Final OOF Performance ---")
final_mse, final_r2 = gwr_trainer_search.evaluate()
if final_mse is not None:
    print(f"Final OOF Mean Squared Error (after search): {final_mse:.4f}")
    print(f"Final OOF R-squared (after search): {final_r2:.4f}")

final_oof_predictions_df = gwr_trainer_search.prediction()
print("\nFinal OOF Predictions Head:")
print(final_oof_predictions_df.head())


--- Step 2.1: Searching for optimal k_nearest in range (100, 500, 50) ---
--- Starting Optimal k_nearest Search ---

--- Testing k_nearest = 100 ---
Mode: Lightweight search. Using 100 cluster centers.


D:\Anaconda\Path\envs\py390\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


TypeError: '<' not supported between instances of 'NoneType' and 'float'

In [49]:
## 直接指定带宽

bandwidth_list = []
cluster_num_list = []
r2_list_true = []
mse_list_true = []
r2_list_interpol = []
mse_list_interpol = []
y_pred_num_list = []
period_list = []

for bandwdith in range(125, 2376, 125):
    for cluster_num in range(50, 501, 50):

        start_time = time.time()
        gwxgb_trainer = GeoWeightedXGBoostTrainer(data=X,
                                                    target=y,
                                                    locations=coords,
                                                    n_jobs=8, 
                                                    use_full_sample = False,
                                                    k_nearest=bandwdith, 
                                                    n_clusters=cluster_num)                          
        gwxgb_trainer.fit()
        local_y_hat, y_hat = gwxgb_trainer.predict()
        mse, r2 = gwxgb_trainer.evaluate_test_set()
        bandwidth_list.append(bandwdith)
        cluster_num_list.append(cluster_num)
        y_pred_num_list.append(len(y_hat))
        r2_list_true.append(r2)
        mse_list_true.append(mse)
            
        if len(y_hat) < 2500:
            y_pred_full = spatial_interpolator_fill(y_hat, coords)
            r2_list_interpol.append(r2_score(y_pred_full, y))
            mse_list_interpol.append(mean_squared_error(y_pred_full, y))
        else:
            r2_list_interpol.append(r2)
            mse_list_interpol.append(mse)

        end_time = time.time()
        period = end_time - start_time
        period_list.append(period)


Starting model fitting with use_full_sample=False and k_nearest=125...
Using 50 cluster centers.
Finding 125 nearest neighbors for each of the 50 centers...


D:\Anaconda\Path\envs\py390\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Evaluating using aggregated test-set predictions on 1466 unique samples...

Starting model fitting with use_full_sample=False and k_nearest=125...


D:\Anaconda\Path\envs\py390\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Using 100 cluster centers.
Finding 125 nearest neighbors for each of the 100 centers...
Evaluating using aggregated test-set predictions on 2068 unique samples...

Starting model fitting with use_full_sample=False and k_nearest=125...


...

Using 400 cluster centers.
Finding 2375 nearest neighbors for each of the 400 centers...
Evaluating using aggregated test-set predictions on 2500 unique samples...

Starting model fitting with use_full_sample=False and k_nearest=2375...


D:\Anaconda\Path\envs\py390\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Using 450 cluster centers.
Finding 2375 nearest neighbors for each of the 450 centers...
Evaluating using aggregated test-set predictions on 2500 unique samples...

Starting model fitting with use_full_sample=False and k_nearest=2375...


D:\Anaconda\Path\envs\py390\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Using 500 cluster centers.
Finding 2375 nearest neighbors for each of the 500 centers...
Evaluating using aggregated test-set predictions on 2500 unique samples...


In [67]:
bw_cluster_df = pd.DataFrame({'bw': bandwidth_list, 'cluster_num': cluster_num_list,
                              'r2_true': r2_list_true, 
                              'r2_interpol': r2_list_interpol, 'mse_interpol': mse_list_interpol, 
                              'y_pred_len': y_pred_num_list, 'period': period_list})


In [70]:
bw_cluster_df1 = bw_cluster_df.copy()

,bw,cluster_num,r2_true,r2_interpol,mse_interpol,y_pred_len,period
4,125,250,0.823789,0.696866,1.442745,2455,33.721497
14,250,250,0.820539,0.719291,1.369410,2495,35.939648
24,375,250,0.810211,0.810211,1.439009,2500,38.611629
34,500,250,0.788175,0.788175,1.606092,2500,39.987272
44,625,250,0.772100,0.772100,1.727971,2500,40.871955
54,750,250,0.756468,0.756468,1.846492,2500,41.432457
64,875,250,0.743021,0.743021,1.948450,2500,41.882589
74,1000,250,0.725556,0.725556,2.080872,2500,42.992430
84,1125,250,0.714403,0.714403,2.165436,2500,42.926432
94,1250,250,0.701571,0.701571,2.262734,2500,42.937742


In [71]:
bw_cluster_df_next = bw_cluster_df[bw_cluster_df['y_pred_len'] < 2500]

bws = bw_cluster_df_next['bw'].values
clusters = bw_cluster_df_next['cluster_num'].values

In [72]:
bws

array([ 125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  250,
        250,  250,  250,  250,  250,  375,  375,  375,  500,  500,  500,
        500,  625,  625,  750,  750,  875,  875, 1000, 1125, 1250, 1375,
       1500, 1625], dtype=int64)

In [73]:
clusters

array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500,  50, 100, 150,
       200, 250, 300,  50, 100, 150,  50, 100, 150, 200,  50, 100,  50,
       100,  50, 100,  50,  50,  50,  50,  50,  50], dtype=int64)

In [12]:
y_hat_500 = local_y_hat.groupby(['index'])['predy'].mean().reset_index()
y_hat_500

,index,predy
0,1,2.702500
1,3,2.262207
2,4,-0.771560
3,5,4.795794
4,6,1.793661
...,...,...
2063,2494,2.805756
2064,2495,3.990550
2065,2496,4.969280
2066,2498,0.311182


In [40]:
y_hat_df = pd.DataFrame(y_hat).reset_index()
y_hat_df.columns = ['ori_idx', 'predy']
# y_hat_df['pred_avg'] = y_hat_500['predy']

y_hat_df

,index,predy
0,0,-0.742861
1,1,3.550641
2,2,1.701274
3,3,4.118545
4,4,0.030738
...,...,...
2495,2495,3.682503
2496,2496,3.352097
2497,2497,1.251574
2498,2498,-0.165576


In [14]:
obtained_index = y_hat_df['ori_idx'].values

print(r2_score(y[obtained_index], y_hat_df['pred_avg']))

0.7692370739875727


In [15]:
y_pred_full = np.full(2500, np.nan)

obtained_index = pd.DataFrame(y_hat).reset_index()['index'].values
y_pred_full[obtained_index] = pd.DataFrame(y_hat).reset_index()['predy'].values
    
mask_known = ~np.isnan(y_pred_full)

y_train = y_pred_full[mask_known]

r2_known = r2_score(y[mask_known], y_train)
mse_know = mean_squared_error(y[mask_known], y_train)

print(r2_known, mse_know)

0.7692370739875727 1.6611380506818294


In [17]:
xgb_trainer = XGBoostTrainer_1(n_splits=5, random_state=42, n_jobs=4)
final_model, _, _, _, _ = xgb_trainer.calcu_oof_and_shap(X[mask_known], y_train, tune=True, max_evals=50)
    
# 对缺失样本进行预测
mask_missing = np.isnan(y_pred_full)
X_missing = X[mask_missing]
y_missing_pred = final_model.predict(xgb.DMatrix(X_missing))

100%|████████████████████████████████████████████████| 50/50 [00:39<00:00,  1.26trial/s, best loss: 1.1204374465094815]

Best parameters found:
{'max_depth': 3, 'learning_rate': 0.036508007159701795, 'subsample': 0.8419736360931015, 'colsample_bytree': 0.8469520993779159, 'min_child_weight': 4, 'gamma': 0.05022181386914993, 'reg_alpha': 0.12722975116876362, 'reg_lambda': 0.384758713405933, 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'random_state': 42}

Starting 5-Fold CV for OOF predictions (parallel)...

--- Training Final Model on Full Data ---
Final model training complete.


In [18]:
r2_missing = r2_score(y[mask_missing], y_missing_pred)
mse_missing = mean_squared_error(y[mask_missing], y_missing_pred)

print(r2_missing, mse_missing)

0.6085111675863619 2.7587029830337135


In [19]:
y_pred_full[mask_missing] = y_missing_pred


In [21]:
mean_squared_error(y_pred_full, y)

1.8507972709922345

0.5795553393859016

In [49]:
r2_score(y_pred_full, y)

0.5795553393859016

In [42]:
mask_missing

array([ True, False,  True, ...,  True, False, False])

In [45]:
y_pred_full[:6]

array([        nan, -2.4983878 ,         nan,  2.69457885,  4.70055883,
        7.75759537])

In [40]:
data = pd.read_csv(r'E:\PaperCode\S-MGXGB\SimuResults_S2_1\data_1.csv')

data_mean = data.groupby('lng')['y'].mean()

data_mean_df = pd.DataFrame(data_mean)

data_mean_df1 = data_mean_df.reset_index()
data_mean_df1.head(5)

,lng,y
0,0.0,1.662997
1,1.0,1.915145
2,2.0,1.340030
3,3.0,1.793759
4,4.0,1.026859


## Model evaluation

In [ ]:
print("RMSE: ", np.sqrt(mean_squared_error(y, y_hat)))
print("adj_R2: ", adjusted_r2_score(y, y_hat, 4))

## SHAP calculation

In [ ]:
test_indices_list = gwxgb_trainer.test_indices_list
interpreter = GeoWeightedXGBoostInterpreter(gwxgb_trainer, use_shap=True)

shap_values_df = pd.DataFrame()
for i, model in enumerate(gwxgb_trainer.models):
    test_indices = test_indices_list[i]
    X_test = self.data[test_indices]
    shap_values = interpreter.calculate_shap_values(model, X_test)
    shap_values_df_i = pd.DataFrame(shap_values, columns=[f"shap_{j}" for j in range(shap_values.shape[1])])
    shap_values_df_i['original_index'] = test_indices
    shap_values_df = pd.concat([shap_values_df, shap_values_df_i], ignore_index=True) ## local
gwxgb_shap_global = shap_values_df.groupby('original_index')[[f"shap_{j}" for j in range(shap_values.shape[1])]].mean().reset_index() ## global

global_shap_xs = np.array(gwxgb_shap_global[['shap_0', 'shap_1', 'shap_2', 'shap_3']]).reshape(2500, 4)
gwxgb_shap_local = shap_values_df.copy()

## Plotting

### Non-linear

In [ ]:
## global relationships
locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-3.2, -0.6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                global_shap_xs[:, i],
                c = 'none',
                edgecolors = 'darkred',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(global_shap_xs[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['GWXGB curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'gwxgb_X{i+1}_nl.jpg', 
                dpi = 300)
    
    plt.close()

In [ ]:
## local relationships

point_id = 'the point id'
vmin = point_id * optimal_k * 0.3
vmax = (point_id+1) * optimal_k * 0.3

for i in range(4):
    nearest_indices = gwxgb_shap_local[vmin:vmax].original_index
    point_1_shap = gwxgb_shap_local[vmin:vmax].loc[:, f'shap_{i}']
    point_1_x3 = X[nearest_indices, i]
    f = fs[[nearest_indices], i].reshape(-1)

    fig = plt.figure(figsize=(8, 6))

    plt.scatter(point_1_x3, 
                point_1_shap,
                c = 'none',
                edgecolors = 'green',
                alpha=0.6, 
                s = 18)

    plt.scatter(point_1_x3, 
               f,
               edgecolors = 'k',
               alpha=0.6, 
               s = 3)

    plt.legend(['GWXGB curve', 'Actual curve'])

    r2 = r2_score(point_1_shap, f)
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})
    
    plt.legend(['GWXGB curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'gwxgb_X{i+1}_nl_local.jpg', 
                dpi = 300)
    
    plt.close()

### Spatial

In [ ]:
vmins = [1.8, 0.5, -3, -2.25]
vmaxs = [2.2, 3, 3, 2.25]

for i in range(5):

    fig = plt.figure(figsize = (8, 8))

    plot_1(moving_wind(global_shap_xs[:, i], X[:, i]), fr'GWXGB: $f_{i+1} $', vmin = vmins[i], vmax = vmaxs[i])
    
    expected = fs[:, i] / X[:, i].reshape(-1)
    predicted = moving_wind(global_shap_xs[:, i], X[:, i]).reshape(-1)
    cosine_sim = np.dot(predicted, expected)/(np.linalg.norm(predicted) * np.linalg.norm(expected))
    
    print(f'the cosine similarity index for x_{i+1} is {cosine_sim}')

    plt.savefig(save_path + '\\' + fr'gwxgb_X{i+1}_s.jpg', 
                dpi = 300)

# MGWR

## Train model

In [ ]:
from mgwr.gwr import GWR, MGWR                                                   
from mgwr.sel_bw import Sel_BW
import time

In [ ]:
sel = Sel_BW(coords, y, X, multi=True, constant=True, kernel='bisquare', fixed=False)
bws = sel.search(verbose=False)
result = MGWR(coords, y, X, selector=sel,constant=True, kernel='bisquare', fixed=False).fit()


In [ ]:
bws

## Model evaluation

In [ ]:
print("RMSE: ", np.sqrt(mean_squared_error(y, result.predy)))
print("adj_R2: ", adjusted_r2_score(y, result.predy, 4))

## Plotting

In [ ]:
params = result.params

### Non-linear

In [ ]:
## global relationships
locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-3.2, -0.6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                params[:, i+1] * X[:, i],
                c = 'none',
                edgecolors = 'darkred',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(params[:, i+1] * X[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['MGWR curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\beta_{i+1} X_{i+1}$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'mgwr_X{i+1}_nl.jpg', 
                dpi = 300)
    
    plt.close()

In [ ]:
## local relationships

locs_X = [0.5, 2, 0.5, 0.5]
locs_Y = [-3.2, -0.6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    
    n_neighbors = int(bws[i+1]) 
    nn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto', metric='euclidean')
    nn.fit(coords)
    distances, selected_indices_list = nn.kneighbors(coords)

    nearest_indices = selected_indices_list[point_id]
    
    b = result.params[:, i+1][point_id]
    f = fs[[nearest_indices], i].reshape(-1)
    point_1_x3 = X[nearest_indices, i]
    
    fig = plt.figure(figsize=(8, 6))

    plt.scatter(point_1_x3, 
                b*point_1_x3,
                c = 'none',
                edgecolors = 'green',
                alpha=0.6, 
                s = 18)

    plt.scatter(point_1_x3, 
               f,
               edgecolors = 'k',
               alpha=0.6, 
               s = 3)

    plt.legend(['MGWR curve', 'Actual curve'])

    r2 = r2_score(b*point_1_x3, f)
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})
    
    plt.legend(['MGWR curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\beta_{i+1} X_{i+1}$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'mgwr_X{i+1}_nl_local.jpg', 
                dpi = 300)
    
    plt.close()

### Spatial

In [ ]:
vmins = [1.8, 0.5, -3, -2.25]
vmaxs = [2.2, 3, 3, 2.25]

for i in range(4):

    fig = plt.figure(figsize = (8, 8))

    plot_1(result.params[:, i+1], fr'MGWR: $f_{i+1} $', vmin = vmins[i], vmax = vmaxs[i])
    
    expected = fs[:, i] / X[:, i].reshape(-1)
    predicted = result.params[:, i+1].reshape(-1)
    cosine_sim = np.dot(predicted, expected)/(np.linalg.norm(predicted) * np.linalg.norm(expected))
    
    print(f'the cosine similarity index for x_{i+1} is {cosine_sim}')

    plt.savefig(save_path + '\\' + fr'mgwr_X{i+1}_s.jpg', 
                dpi = 300)

# M-GWXGB


**1. we first generate the results from GWXGB under 19 bandwidth, named like 'shap_values_bw{bandwidth}_global.csv' and 'shap_values_bw{bandwidth}_local.csv' under your path, where the file ending with "local.csv" include the shapley values from all sub-models. and the file ending with "global.csv" average the output of "local.csv" file by the sample id**  
**2. we permutate the shap value to generate the best prediction combination**  
**3. use LtoG method to identify the local and global estimator**  
**4. set the prediction combination to the initialization and use M-GWXGB to iteratively fit data**

## LtoG approach

In [ ]:
## generate prediction for each variable under each bandwidth
bandwidths = [i for i in range(125, 2376, 125)]
save_dir = r'....\\GWXGB_full'

mul_gwxgb = M_GWXGB_Initial(data=X, target=y, locations=coords, bandwidths=bandwidths, 
                             use_shap=True, 
                             n_jobs=20, 
                             shap_save_dir=save_dir)

mul_gwxgb.fit()

In [ ]:
## permutation

input_files = []
for bandwidth in range(125, 2376, 125):
    input_files.append(save_dir + '\\' f'shap_values_bw{bandwidth}_global.csv')

opt_bandwidth = OptimizedBandwidth(X, y, input_files, shap_save_dir=save_dir)
best_combination = opt_bandwidth.optimize_bandwidths()

bst_gwxgb = best_combination['bst_shap_values']

In [ ]:
## LtoG

gw_shap_values = bst_gwxgb[['shap_0', 'shap_1', 'shap_2','shap_3']]
final_shap_values = gw_shap_values.copy()
model_choice = {}
shap_contributions = {}

y_pred_current = np.sum(final_shap_values.values, axis=1) + np.mean(y)
best_rmse = np.sqrt(mean_squared_error(y, y_pred_current))

print(f"Original RMSE: {best_rmse:.6f}")

for i in range(4):
    shap_names = [f'shap_{j}' for j in range(X.shape[1])]
    shap_names.pop(i)
    y_pred_other = np.sum(final_shap_values[shap_names], axis=1) + np.mean(y)
    y_pred_other = np.array(y_pred_other).reshape(-1, 1)
    y_leave = y - y_pred_other
    X_leave = X[:, i]

    xgb_trainer = XGBoostTrainer_1(n_splits=5, random_state=42, n_jobs=20)
    final_model, oof_preds, oof_rmse, oof_r2, oof_shap = xgb_trainer.calcu_oof_and_shap(X_leave, y_leave, tune=True, max_evals=50)

    temp_shap_values = final_shap_values.copy()
    temp_shap_values[f'shap_{i}'] = oof_preds
    y_pred_new = np.sum(temp_shap_values.values, axis=1) + np.mean(y)
    rmse_new = np.sqrt(mean_squared_error(y, y_pred_new))
    r2_new = r2_score(y, y_pred_new)

    print(f'\nthe {i}_th var')
    print("--- OOF Performance Metrics ---")
    print(f"Current best RMSE: {best_rmse:.6f}")
    print(f"New candidate RMSE: {rmse_new:.6f}")

    var_name = f'shap_{i}'
    if rmse_new < best_rmse:
        print(f'the best model for {i}_th var is xgb')
        final_shap_values[var_name] = oof_preds
        model_choice[var_name] = 'xgb'
        shap_contributions[var_name] = oof_preds
        best_rmse = rmse_new
        y_pred_current = y_pred_new
    else:
        print(f'the best model for {i}_th var is geo')
        model_choice[var_name] = 'geo'
        shap_contributions[var_name] = final_shap_values[var_name].values

print("\nfinal model choice:", model_choice)

## train model

In [ ]:
selected_learner_types = list(model_choice.values())

k_ranges_for_selection = [(125, 2376, 125), (125, 2376, 125), (125, 2376, 125), (125, 2376, 125)]

col_names = [f'shap_{i}' for i in range(4)]
initial_results = pd.DataFrame({
    col: np.array(shap_contributions[col]).flatten()
    for col in shap_contributions
})

shap_names = [f'shap_{j}' for j in range(X.shape[1])]
ini_component_preds = np.array(initial_results[shap_names])

m_gwxgb = M_GXGB(
    data=X,                                                                                                                                                                                                                                                                            
    target=y, 
    locations=coords,
    selected_learner_types=selected_learner_types, 
    k_range=k_ranges_for_selection,
    convergence_tol_percent=1,
    n_jobs=20,
    init_component_predictions=ini_component_preds
    )

m_gwxgb.fit()

# Predict using the fitted M-GWXGB
y_hat_mgwxgb = np.sum(m_gwxgb.component_predictions_, axis = 1) + np.mean(y)
mgwxgb_global_shap = m_gwxgb.component_predictions_

print(m_gwxgb.optimal_bandwidths_)

In [ ]:
mgwxgb_local = pd.DataFrame(gam_gb.local_prediction_df)
mgwxgb_local.columns = ['shap_0', 'index_1', 'shap_1', 'shap_2', 'index_3', 'shap_3']

## Model evaluation

In [ ]:
mgwxgb_rmse = np.sqrt(mean_squared_error(y.ravel(), y_hat_mgwxgb))
mgwxgb_r2 = adjusted_r2_score(y.ravel(), y_hat_mgwxgb, 4)

print(f"RMSE: {mgwxgb_rmse:.4f}")
print(f"Adj_R2: {mgwxgb_r2:.4f}")

## Plotting

### Non-linear

In [ ]:
## global relationships
locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-3.2, -0.6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                mgwxgb_global_shap[:, i],
                c = 'none',
                edgecolors = 'darkred',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(mgwxgb_global_shap[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['M-GWXGB curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'mgwxgb_X{i+1}_nl.jpg', 
                dpi = 300)
    
    plt.close()

In [ ]:
## local

locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-3.2, -0.6, -3.2, -2]

for i in range(4):
    
    fig = plt.figure(figsize=(8, 6))
    if i in range(2, 5):
            
        vmin = bws_mgwxgb[i] * 1
        vmax = bws_mgwxgb[i] * (1+1)
        
        nearest_indices = mgwxgb_local[vmin:vmax].loc[:, f'index_{i}']
        nearest_indices_1 = nearest_indices.astype(int)
        point_1_shap = mgwxgb_local[vmin:vmax].loc[:, f'shap_{i}']
        point_1_x3 = X[nearest_indices_1, i]
        f = fs[[nearest_indices_1], i].reshape(-1)

        plt.scatter(point_1_x3, 
                    point_1_shap,
                    c = 'none',
                    edgecolors = 'green',
                    alpha=0.6, 
                    s = 18)

        plt.scatter(point_1_x3, 
                   f,
                   edgecolors = 'k',
                   alpha=0.6, 
                   s = 3)

        plt.legend(['M-GWXGB curve', 'Actual curve'])

        r2 = r2_score(point_1_shap, f)
        plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

        plt.legend(['M-GWXGB curve', 'Actual curve'], loc = locs[i])

        plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
        plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')

    else:
        plt.scatter(X[:, i], 
                    mgwxgb_global[f'f{i+1}_hat'],
                    c = 'none',
                    edgecolors = 'green',
                    alpha=0.6, 
                    s = 18)

        plt.scatter(X[:, i], 
                    fs[:, i], 
                    c = 'k', 
                    s = 3, 
                    alpha = 0.7)

        r2 = r2_score(mgwxgb_global[f'f{i+1}_hat'], fs[:, i])
        plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

        plt.legend(['M-GWXGB curve', 'Actual curve'], loc = locs[i])

        plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
        plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')

    
    plt.savefig(save_path + '\\' + fr'mgwxgb_X{i+1}_nl_local_1.jpg', 
                dpi = 300)
    
    plt.close()

### Spatial

In [ ]:
vmins = [1.8, 0.5, -3, -2.25]
vmaxs = [2.2, 3, 3, 2.25]

for i in range(5):

    fig = plt.figure(figsize = (8, 8))

    plot_1(moving_wind(mgwxgb_global_shap[:, i], X[:, i]), fr'M-GWXGB: $f_{i+1} $', vmin = vmins[i], vmax = vmaxs[i])
    
    expected = fs[:, i] / X[:, i].reshape(-1)
    predicted = moving_wind(mgwxgb_global_shap[:, i], X[:, i], w = 2).reshape(-1)
    cosine_sim = np.dot(predicted, expected)/(np.linalg.norm(predicted) * np.linalg.norm(expected))
    
    print(f'the cosine similarity index for x_{i+1} is {cosine_sim}')

    plt.savefig(save_path + '\\' + fr'mgwxgb_X{i+1}_s.jpg', 
                dpi = 300)

# GCN

## Train model

In [ ]:
import torch
import itertools
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import kneighbors_graph

In [ ]:
class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

class GCNTrainer:
    def __init__(self, input_dim, n_splits=5, random_state=42):
        self.input_dim = input_dim
        self.n_splits = n_splits
        self.random_state = random_state

        self.best_params = None
        self.best_avg_rmse = float('inf')

        self.cv_scores = []
        self.oof_predictions = None 
        self.oof_shap_values = None
        self.oof_rmse = None
        self.oof_r2 = None

        self.X_scaler = None 
        self.y_scaler = None

    def _train_fold(self, data, train_mask, val_mask, params):
        """Trains and evaluates the model for a single fold using specific parameters."""
        hidden_dim = int(params['hidden_dim'])
        lr = params['lr']
        num_epochs = int(params['num_epochs'])
        dropout_rate = params['dropout_rate']
        weight_decay = params['weight_decay']

        model = GCNModel(
            input_dim=self.input_dim,
            hidden_dim=hidden_dim,
            output_dim=1,
            dropout_rate=dropout_rate
        )
        optimizer = optim.Adam(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay
        )
        criterion = nn.MSELoss() 

        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            out = model(data)[train_mask]
            loss = criterion(out, data.y[train_mask].view(-1, 1)) # Ensure target is 2D
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_out_scaled = model(data)[val_mask].cpu().numpy()

            y_val_scaled_np = data.y[val_mask].cpu().numpy().reshape(-1, 1) 

            val_out_original = self.y_scaler.inverse_transform(val_out_scaled)
            y_val_original = self.y_scaler.inverse_transform(y_val_scaled_np)

            val_rmse = np.sqrt(mean_squared_error(y_val_original, val_out_original))

        return model, val_rmse, val_out_scaled

    def _run_cv_for_objective(self, data_for_trial, params):
        """Runs K-Fold CV for a specific set of hyperparameters and graph structure."""
        kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        fold_rmses = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(data_for_trial.x.shape[0]))):
            train_mask = torch.zeros(data_for_trial.x.shape[0], dtype=torch.bool)
            val_mask = torch.zeros(data_for_trial.x.shape[0], dtype=torch.bool)
            train_mask[train_idx] = True
            val_mask[val_idx] = True

            try:
                 _, fold_rmse, _ = self._train_fold(data_for_trial, train_mask, val_mask, params)
                 fold_rmses.append(fold_rmse)
            except Exception as e:
                 print(f"Error training fold {fold+1} with params {params}: {e}")
                 return float('inf')

        if not fold_rmses:
            return float('inf')

        avg_rmse = np.mean(fold_rmses)
        return avg_rmse

    def objective(self, params, X_tensor_scaled, y_tensor, coords, X_scaler, y_scaler):
        """Objective function for Hyperopt Bayesian Optimization."""
        self.X_scaler = X_scaler
        self.y_scaler = y_scaler

        n_neighbors = int(params['n_neighbors'])
        try:
            graph = kneighbors_graph(coords, n_neighbors=n_neighbors, mode="connectivity", include_self=False)
            edge_index = torch.tensor(np.array(graph.nonzero()), dtype=torch.long).contiguous() 
            data_for_trial = Data(x=X_tensor_scaled, edge_index=edge_index, y=y_tensor)
        
        except Exception as e:
             print(f"Error building graph with n_neighbors={n_neighbors}: {e}")
             return {'loss': float('inf'), 'status': STATUS_OK}

        avg_cv_rmse = self._run_cv_for_objective(data_for_trial, params)

        return {'loss': avg_cv_rmse, 'status': STATUS_OK}

    def bayes_optimize(self, X_tensor_scaled, y_tensor, coords, X_scaler, y_scaler, search_space, max_evals=50):
        self.X_scaler = X_scaler
        self.y_scaler = y_scaler

        trials = Trials()

        best = fmin(
            fn=lambda params: self.objective(params, X_tensor_scaled, y_tensor, coords, X_scaler, y_scaler),
            space=search_space,
            algo=tpe.suggest,
            max_evals=max_evals,
            trials=trials,
            rstate=np.random.default_rng(self.random_state)
        )

        self.best_params = {
            'hidden_dim': int(best['hidden_dim']),
            'lr': best['lr'],
            'num_epochs': int(best['num_epochs']),
            'dropout_rate': best['dropout_rate'],
            'weight_decay': best['weight_decay'],
            'n_neighbors': int(best['n_neighbors'])
        }

        print("\nRe-evaluating best parameters to get final CV RMSE...")
        best_n_neighbors = int(self.best_params['n_neighbors'])
        graph_best = kneighbors_graph(coords, n_neighbors=best_n_neighbors, mode="connectivity", include_self=False)
        edge_index_best = torch.tensor(np.array(graph_best.nonzero()), dtype=torch.long).contiguous()
        data_best_graph = Data(x=X_tensor_scaled, edge_index=edge_index_best, y=y_tensor)

        self.X_scaler = X_scaler
        self.y_scaler = y_scaler
        self.best_avg_rmse = self._run_cv_for_objective(data_best_graph, self.best_params)

        print("\nBayesian Optimization complete.")
        print(f"Best Hyperparameters found: {self.best_params}")
        print(f"Best Average CV RMSE from Re-evaluation: {self.best_avg_rmse:.4f}")
        return self.best_params, self.best_avg_rmse

    def predict_for_shap(self, x_np_batch, model, X_scaler, y_scaler, n_neighbors):

        model.eval()
        x_np_batch = x_np_batch.astype(np.float32)

        try:
             coords_batch = x_np_batch[:, :2]
             if coords_batch.shape[0] <= n_neighbors:
                return np.full(x_np_batch.shape[0], np.nan)

             graph = kneighbors_graph(coords_batch, n_neighbors=n_neighbors, mode="connectivity", include_self=False)
             edge_index_batch = torch.tensor(np.array(graph.nonzero()), dtype=torch.long).contiguous() # Ensure contiguous
        except Exception as e:
             print(f"Unexpected error building graph for SHAP with n_neighbors={n_neighbors}, batch shape {x_np_batch.shape}: {e}")
             return np.full(x_np_batch.shape[0], np.nan)

        x_tensor_scaled = torch.tensor(X_scaler.transform(x_np_batch), dtype=torch.float32)

        new_data_batch = Data(x=x_tensor_scaled, edge_index=edge_index_batch)

        with torch.no_grad():
            predictions_scaled = model(new_data_batch).detach().cpu().numpy() # Move to CPU

            predictions_original = y_scaler.inverse_transform(predictions_scaled)

        return predictions_original.flatten()

    def calculate_oof_and_shap(self, X_tensor_scaled, y_tensor, coords, X_original_np, y_original_np, X_scaler, y_scaler, feature_names):

        if self.best_params is None:
            raise ValueError("Best parameters not set. Run bayes_optimize first.")

        # Store scalers
        self.X_scaler = X_scaler
        self.y_scaler = y_scaler

        best_n_neighbors = int(self.best_params['n_neighbors'])
        print(f"\nBuilding final graph for OOF run with best n_neighbors: {best_n_neighbors}")
        try:
             graph = kneighbors_graph(coords, n_neighbors=best_n_neighbors, mode="connectivity", include_self=False)
             edge_index_final = torch.tensor(np.array(graph.nonzero()), dtype=torch.long).contiguous() # Ensure contiguous
             data_final = Data(x=X_tensor_scaled, edge_index=edge_index_final, y=y_tensor)
             print(f"Final graph built with {edge_index_final.shape[1]} edges.")
        except Exception as e:
            print(f"FATAL ERROR: Could not build final graph with n_neighbors={best_n_neighbors}: {e}")
            self.oof_rmse, self.oof_r2 = np.nan, np.nan
            return np.full(X_original_np.shape[0], np.nan), np.full(X_original_np.shape, np.nan), self.oof_rmse, self.oof_r2, np.zeros(X_original_np.shape[0], dtype=bool)

        print(f"\nStarting {self.n_splits}-Fold CV for OOF predictions and SHAP values using best params...")

        kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)

        self.oof_predictions = np.zeros(X_original_np.shape[0])
        self.oof_shap_values = np.zeros(X_original_np.shape)
        self.cv_scores = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(data_final.x.shape[0]))):
            print(f"\n--- Processing Fold {fold+1}/{self.n_splits} ---")

            train_mask = torch.zeros(data_final.x.shape[0], dtype=torch.bool)
            val_mask = torch.zeros(data_final.x.shape[0], dtype=torch.bool)
            train_mask[train_idx] = True
            val_mask[val_idx] = True

            fold_model, fold_val_rmse, val_out_scaled = self._train_fold(data_final, train_mask, val_mask, self.best_params)
            self.cv_scores.append(fold_val_rmse)
            print(f"Fold {fold+1} Validation RMSE: {fold_val_rmse:.4f}")

            val_preds_original = self.y_scaler.inverse_transform(val_out_scaled)
            self.oof_predictions[val_idx] = val_preds_original.flatten() # Ensure 1D

            X_val_fold_np = X_original_np[val_idx] 

            if len(X_val_fold_np) > 0:
                # Instantiate SHAP Explainer
                print(f"Calculating SHAP values for {len(val_idx)} validation nodes in Fold {fold+1} using best n_neighbors={best_n_neighbors}...")
                try:
                     explainer = shap.Explainer(
                         lambda x_batch: self.predict_for_shap(x_batch, fold_model, self.X_scaler, self.y_scaler, best_n_neighbors),
                         X_original_np[train_idx], 
                         feature_names=feature_names 
                     )

                     shap_values_fold = explainer(X_val_fold_np).values
                     print(f"Finished SHAP calculation for Fold {fold+1}.")

                     self.oof_shap_values[val_idx, :] = shap_values_fold

                except Exception as e:
                     print(f"SHAP calculation failed for Fold {fold+1}: {e}")
                     print("Storing NaNs for SHAP values in this fold.")
                     self.oof_shap_values[val_idx, :] = np.full_like(self.oof_shap_values[val_idx, :], np.nan)

            else:
                 print(f"  Fold {fold+1} validation set is empty. Skipping SHAP calculation.")

        print("\n5-Fold CV and SHAP calculation complete.")

        valid_nodes_for_oof = ~np.isnan(self.oof_predictions)
        shap_row_has_nan = np.isnan(self.oof_shap_values).any(axis=1)
        valid_nodes_for_oof = valid_nodes_for_oof & (~shap_row_has_nan)


        if valid_nodes_for_oof.sum() > 0:
             y_valid = y_original_np[valid_nodes_for_oof]
             oof_preds_valid = self.oof_predictions[valid_nodes_for_oof]
             oof_shap_valid = self.oof_shap_values[valid_nodes_for_oof]
             X_combined_valid = X_original_np[valid_nodes_for_oof]

             # Calculate overall OOF metrics on valid nodes
             self.oof_rmse = np.sqrt(mean_squared_error(y_valid, oof_preds_valid))
             self.oof_r2 = r2_score(y_valid, oof_preds_valid)

             print(f"\n--- Overall OOF Results (on {valid_nodes_for_oof.sum()} valid nodes) ---")
             print(f"Overall OOF RMSE: {self.oof_rmse:.4f}")
             print(f"Overall OOF R2: {self.oof_r2:.4f}")
             print(f"Average Fold Validation RMSE: {np.mean(self.cv_scores):.4f} (Std: {np.std(self.cv_scores):.4f})")
             return self.oof_predictions, self.oof_shap_values, self.oof_rmse, self.oof_r2, valid_nodes_for_oof, X_combined_valid
        else:
            print("No valid OOF predictions or SHAP values were computed across folds.")
            self.oof_rmse, self.oof_r2 = np.nan, np.nan
            return self.oof_predictions, self.oof_shap_values, self.oof_rmse, self.oof_r2, valid_nodes_for_oof, None # Return None for X_combined_valid if no valid nodes

In [ ]:
# Standardize features (fit on full X)

feature_names = [f'X{i+1}' for i in range(X.shape[1])]
X_combined = pd.DataFrame(X, columns=feature_names)

X_scaler = StandardScaler()
X_combined_standardized = X_scaler.fit_transform(X)

# Standardize target separately
y_scaler = StandardScaler()
y_standardized = y_scaler.fit_transform(y.reshape(-1, 1))

X_tensor_scaled = torch.tensor(X_combined_standardized, dtype=torch.float32)
y_tensor = torch.tensor(y_standardized, dtype=torch.float32)

In [ ]:
search_space = {
    'hidden_dim': hp.quniform('hidden_dim', 16, 128, 1), 
    'lr': hp.loguniform('lr', np.log(0.00005), np.log(0.01)),
    'num_epochs': hp.quniform('num_epochs', 100, 600, 1), 
    'dropout_rate': hp.uniform('dropout_rate', 0.1, 0.6),
    'weight_decay': hp.loguniform('weight_decay', np.log(1e-5), np.log(1e-3)),
    'n_neighbors': hp.quniform('n_neighbors', 10, 40, 1) 
}

In [ ]:
print("\nInstantiating GCN Trainer...")
gcn_trainer = GCNTrainer(
    input_dim=X_combined_standardized.shape[1],
    n_splits=5,
)

print("\nStarting Bayesian Optimization...")
start_time_bo = time.time()
max_optimization_evals = 50 

best_params, best_avg_rmse_opt = gcn_trainer.bayes_optimize(
    X_tensor_scaled, y_tensor, coords, X_scaler, y_scaler, search_space, max_evals=max_optimization_evals
)
end_time_bo = time.time()
print(f"Bayesian Optimization ({max_optimization_evals} evals) took {end_time_bo - start_time_bo:.2f} seconds.")

start_time_oof_shap = time.time()
oof_predictions, oof_shap_values, oof_rmse, oof_r2, valid_nodes_for_oof, X_combined_valid = gcn_trainer.calculate_oof_and_shap(
    X_tensor_scaled, y_tensor, coords, X, y, X_scaler, y_scaler, feature_names
)
end_time_oof_shap = time.time()
print(f"OOF prediction and SHAP calculation took {end_time_oof_shap - start_time_oof_shap:.2f} seconds.")

## Model evaluation

In [ ]:
print("RMSE: ", oof_rmse)
print("adj_R2: ", adjusted_r2_score(oof_predictions, y, 4))

## Plotting

### Non-linear

In [ ]:
## global
locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-7, -6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                oof_shap_values[:, i],
                c = 'none',
                edgecolors = 'darkred',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(oof_shap_values[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['GCN curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'gcn_X{i+1}_nl.jpg', 
                dpi = 300)
    
    plt.close()

In [ ]:
## local

x_test = np.random.uniform(-1.5, 1.5, 2500)
f_test = - x_test ** 3

for i in range(4):
    plt.figure(figsize=(9, 7))
    b = oof_shap_values[:, i][point_id] / X[:, i][point_id]
    plt.scatter(x_test, 
                b*x_test,
                c = 'none',
                edgecolors = 'green',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(x_test, 
                f_test, 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(b*x_test, f_test)
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['GCN curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\phi_{i+1} (X_{i+1})$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'gcn_X{i+1}_nl_local_1.jpg', 
                dpi = 300)
    
    plt.close()

### Spatial

In [ ]:
vmins = [1.8, 0.5, -3, -2.25]
vmaxs = [2.2, 3, 3, 2.25]

for i in range(5):

    fig = plt.figure(figsize = (8, 8))

    fig = plt.figure(figsize = (8, 8))

    plot_1(moving_wind(oof_shap_values[:, i], X[:, i]), fr'GCN: $f_{i+1} $', vmin = vmins[i], vmax = vmaxs[i])
    
    expected = fs[:, i] / X[:, i].reshape(-1)
    predicted = moving_wind(oof_shap_values[:, i], X[:, i]).reshape(-1)
    cosine_sim = np.dot(predicted, expected)/(np.linalg.norm(predicted) * np.linalg.norm(expected))
    
    print(f'the cosine similarity index for x_{i+1} is {cosine_sim}')

    plt.savefig(save_path + '\\' + fr'gcn_X{i+1}_s.jpg', 
                dpi = 300)
    
    plt.close()

# GGP-GAM

## GGP_results

In [ ]:
ggp_results = pd.read_csv(save_dir + '\\' + r'ggp_gam_svc_all_location_results.csv') ## generated by R code

ggp_results.head(5)

## Model evaluation

In [ ]:
y_pred = np.array(ggp_results['predicted_y_svc_all']).reshape(-1, 1)

print("RMSE: ", np.sqrt(mean_squared_error(y, y_pred)))
print("adj_R2: ", adjusted_r2_score(y, result.predy, 4))

## Plotting

In [ ]:
betas = np.array(ggp_results[['estimated_beta_X1', 'estimated_beta_X2', 'estimated_beta_X3', 'estimated_beta_X4']]).reshape(2500, 4)


### Non-linear

In [ ]:
## global relationships
locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-3.2, -0.6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                betas[:, i] * X[:, i],
                c = 'none',
                edgecolors = 'darkred',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(betas[:, i] * X[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['GGP-GAM curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\beta_{i+1} X_{i+1}$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'ggp_X{i+1}_nl.jpg', 
                dpi = 300)
    
    plt.close()

In [ ]:
## local relationships

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                betas[:, i] * X[:, i],
                c = 'none',
                edgecolors = 'green',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(betas[:, i] * X[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['GGP-GAM curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\beta_{i+1} X_{i+1}$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'ggp_X{i+1}_nl_local.jpg', 
                dpi = 300)
    
    plt.close()

### Spatial

In [ ]:
vmins = [1.8, 0.5, -3, -2.25]
vmaxs = [2.2, 3, 3, 2.25]

for i in range(4):

    fig = plt.figure(figsize = (8, 8))

    plot_1(betas[:, i], fr'GGP-GAM: $f_{i+1} $', vmin = vmins[i], vmax = vmaxs[i])
    
    expected = fs[:, i] / X[:, i].reshape(-1)
    predicted = betas[:, i].reshape(-1)
    cosine_sim = np.dot(predicted, expected)/(np.linalg.norm(predicted) * np.linalg.norm(expected))
    
    print(f'the cosine similarity index for x_{i+1} is {cosine_sim}')

    plt.savefig(save_path + '\\' + fr'ggp_X{i+1}_s.jpg', 
                dpi = 300)

# S-GWR

In [ ]:
sgwr_results = pd.read_csv(save_path + '\\' + r'S_GWR_listwise.csv')
sgwr_results[' est_x1'] = 2.015515   ## extract from the "txt" file generated by the GWR 4.0 Software
sgwr_results[' est_x2'] = 0.540782
sgwr_results[' est_x3'] = 1.332049
sgwr_results.head()

## Model evaluation

In [ ]:
y_pred = np.array(sgwr_results[' yhat']).reshape(-1, 1)

print("RMSE: ", np.sqrt(mean_squared_error(y, y_pred)))
print("adj_R2: ", adjusted_r2_score(y, y_pred, 4))

## Plotting

In [ ]:
betas = np.array(sgwr_results[[' est_x1', ' est_x2', ' est_x3', ' est_x4']]).reshape(2500, 4)

### 非线性

In [ ]:
## global relationships
locs_X = [0.5, 2.2, 0.5, -0.5]
locs_Y = [-3.2, -0.6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

for i in range(4):
    plt.figure(figsize=(9, 7))

    plt.scatter(X[:, i], 
                betas[:, i] * X[:, i],
                c = 'none',
                edgecolors = 'darkred',
                alpha=0.6, 
                s = 18)
    
    plt.scatter(X[:, i], 
                fs[:, i], 
                c = 'k', 
                s = 3, 
                alpha = 0.7)
    
    r2 = r2_score(params[:, i] * X[:, i], fs[:, i])
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})

    plt.legend(['S-GWR curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\beta_{i+1} X_{i+1}$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'sgwr_X{i+1}_nl.jpg', 
                dpi = 300)
    
    plt.close()

In [ ]:
## local relationships

locs_X = [0.5, 2, 0.5, 0.5]
locs_Y = [-3.2, -0.6, -3.2, -3]
locs = ['upper left', 'upper left', 'upper center','upper center', 'upper left']

n_neighbors = 116 
nn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto', metric='euclidean')
nn.fit(coords)
distances, selected_indices_list = nn.kneighbors(coords)

nearest_indices = selected_indices_list[point_id]

for i in range(4):
        
    b = betas[:, i][point_id]
    f = fs[[nearest_indices], i].reshape(-1)
    point_1_x3 = X[nearest_indices, i]
    
    fig = plt.figure(figsize=(8, 6))

    plt.scatter(point_1_x3, 
                b*point_1_x3,
                c = 'none',
                edgecolors = 'green',
                alpha=0.6, 
                s = 18)

    plt.scatter(point_1_x3, 
               f,
               edgecolors = 'k',
               alpha=0.6, 
               s = 3)

    plt.legend(['S-GWR curve', 'Actual curve'])

    r2 = r2_score(b*point_1_x3, f)
    plt.text(locs_X[i], locs_Y[i], '$R^2$: %s' %(round(r2, 3)), fontdict={'size':32})
    
    plt.legend(['S-GWR curve', 'Actual curve'], loc = locs[i])

    plt.ylabel(fr'$\beta_{i+1} X_{i+1}$')
    plt.xlabel(fr'$\mathrm{{X}}_{i+1}$')


    plt.savefig(save_path + '\\' + fr'sgwr_X{i+1}_nl_local.jpg', 
                dpi = 300)
    
    plt.close()

### Spatial

In [ ]:
vmins = [1.8, 0.5, -3, -2.25]
vmaxs = [2.2, 3, 3, 2.25]

for i in range(4):

    fig = plt.figure(figsize = (8, 8))

    plot_1(betas[:, i], fr'S-GWR: $f_{i+1} $', vmin = vmins[i], vmax = vmaxs[i])
    
    expected = fs[:, i] / X[:, i].reshape(-1)
    predicted = betas[:, i].reshape(-1)
    cosine_sim = np.dot(predicted, expected)/(np.linalg.norm(predicted) * np.linalg.norm(expected))
    
    print(f'the cosine similarity index for x_{i+1} is {cosine_sim}')

    plt.savefig(save_path + '\\' + fr'sgwr_X{i+1}_s.jpg', 
                dpi = 300)

In [ ]:

mse_list = []
r2_list = []

for bandwidth in range(125, 2376, 125):

    start_time = time.time()
    geo_trainer = GeoWeightedXGBoostTrainer(
        X, 
        y, 
        coords,
        use_full_sample=False, # 设定最终的训练模式
        n_clusters=100,
        k_nearest=bandwidth,
        n_jobs=-1
    )   

    geo_trainer.fit()

    mse, r2_value = geo_trainer.evaluate()

    mse_list.append(mse)
    r2_list.append(r2_value)

    end_time = time.time()
    period_time = end_time - start_time
    print(period_time)


print(f'mse values: {mse_list}')
print(f'r2 values: {r2_list}')

In [19]:
data = pd.read_csv(r'E:\PaperCode\S-MGXGB\SimuResults_S2_1\data_1.csv')

data.head(5)

,y,x1,x2,x3,x4,lng,lat,f1,f2,f3,f4
0,2.162549,0.336511,-0.019114,-0.294332,0.1,0.0,0.0,0.673021,-0.000000,1.081358e-16,-0.299573
1,0.341259,-0.992791,-0.286502,1.381216,0.1,1.0,0.0,-1.985581,-0.008682,2.627975e-01,-0.299573
2,2.377965,-0.191823,0.178069,0.059899,0.1,2.0,0.0,-0.383646,0.010792,2.297591e-02,-0.299573
3,2.965798,0.807787,0.833617,0.796014,0.1,3.0,0.0,1.615575,0.075783,4.610012e-01,-0.299573
4,1.586662,-0.614024,-1.190700,0.982412,0.1,4.0,0.0,-1.228048,-0.144327,7.623870e-01,-0.299573


In [25]:
data_mean = data.groupby('lng')['x1'].mean()

data_mean_df = pd.DataFrame({'data_mean': data_mean})

data_mean_df_1 = pd.DataFrame(data_mean_df)

In [26]:
data_mean_df_1

,data_mean
lng,
0.0,0.113318
1.0,0.181372
2.0,-0.130977
3.0,0.131323
4.0,-0.070949
5.0,-0.157534
6.0,0.030713
7.0,-0.029396
8.0,-0.017741
